<a href="https://colab.research.google.com/github/Duffye23/Home_Sales/blob/main/home_sales_challenge_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,332 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,047 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,582 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/ubu

In [2]:
# Start a SparkSession
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
sales_df.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# 2. Create a temporary view of the DataFrame.
sales_df.createOrReplaceTempView('sales')


In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""SELECT YEAR(date) AS year, ROUND(AVG(price),2) AS average_price
             FROM sales WHERE bedrooms=4
             GROUP BY year
             ORDER BY year""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
          FROM sales WHERE bedrooms=3 AND bathrooms=3
          GROUP BY date_built
          ORDER BY date_built""").show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
          FROM sales WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000
          GROUP BY date_built ORDER BY date_built""").show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_rating = """ SELECT view AS Rating, ROUND(AVG(price), 2) AS avg_price
                    FROM sales GROUP BY view HAVING avg_price >= 350000
                    ORDER BY view DESC """
spark.sql(view_rating).show()


print("--- %s seconds ---" % (time.time() - start_time))

+------+----------+
|Rating| avg_price|
+------+----------+
|    99|1061201.42|
|    98|1053739.33|
|    97|1129040.15|
|    96|1017815.92|
|    95| 1054325.6|
|    94| 1033536.2|
|    93|1026006.06|
|    92| 970402.55|
|    91|1137372.73|
|    90|1062654.16|
|    89|1107839.15|
|    88|1031719.35|
|    87| 1072285.2|
|    86|1070444.25|
|    85|1056336.74|
|    84|1117233.13|
|    83|1033965.93|
|    82| 1063498.0|
|    81|1053472.79|
|    80| 991767.38|
+------+----------+
only showing top 20 rows

--- 1.1234242916107178 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")


DataFrame[]

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [16]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql(view_rating).show()

print("--- %s seconds ---" % (time.time() - start_time))


+------+----------+
|Rating| avg_price|
+------+----------+
|    99|1061201.42|
|    98|1053739.33|
|    97|1129040.15|
|    96|1017815.92|
|    95| 1054325.6|
|    94| 1033536.2|
|    93|1026006.06|
|    92| 970402.55|
|    91|1137372.73|
|    90|1062654.16|
|    89|1107839.15|
|    88|1031719.35|
|    87| 1072285.2|
|    86|1070444.25|
|    85|1056336.74|
|    84|1117233.13|
|    83|1033965.93|
|    82| 1063498.0|
|    81|1053472.79|
|    80| 991767.38|
+------+----------+
only showing top 20 rows

--- 0.5106847286224365 seconds ---


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sales_df.write.partitionBy("date_built").parquet("date_built_partition", mode="overwrite")

In [19]:
# 11. Read the parquet formatted data.
parquet_sales_df = spark.read.parquet('date_built_partition')

In [20]:
# 12. Create a temporary table for the parquet data.
parquet_sales_df.createOrReplaceGlobalTempView('sales_partition')

In [21]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

view_rating = """ SELECT view AS Rating, ROUND(AVG(price), 2) AS avg_price
                    FROM sales_partition GROUP BY view HAVING avg_price >= 350000
                    ORDER BY view DESC """

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00011801719665527344 seconds ---


In [22]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table sales')

DataFrame[]

In [23]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('sales')


False